In [28]:
import numpy as np

There are eight distinct choice probabilities, one for each of the eight possible states, given by $(a_{it-1}, a_{-it-1}, M_t)$.

We know that, by the properties of the Gumbel distribution,
$$\mathbb{E}\left[\varepsilon(a_i)|x,\sigma^* = 1\right] = \mu + \log\left(1 + \exp(\pi(a_i,x) - \pi(a_{-i},x))\right)$$

In [29]:
beta = .95
lam = 2
delta = 2
gamma = 1.5
mu = .5772
M = np.array([1.,1.5])
B = np.array([.5,.5,.4,.6]).reshape(2,-1)
X = (np.array(np.meshgrid([0,1], [0,1], [1,1.5])).T).reshape(-1,3)
X = X[np.lexsort((X[:,1], X[:,2]))]

In [35]:
# payoff function
def Pi(choice, a_i, a_j, M):
    return(choice * (lam * M - delta * a_j - gamma * (1-a_i)))

# expected payoff
def pi(choice, P_cond):
    a_i = X[:,0]
    M = X[:,2]
    
    P_cond_2 = P_cond.copy()
    P_cond_2[2], P_cond_2[1], P_cond_2[6], P_cond_2[5] = P_cond_2[1], P_cond_2[2], P_cond_2[5], P_cond_2[6]
    
    Pi_val = Pi(choice, a_i, np.ones(8), M) * P_cond_2 + Pi(choice, a_i, np.zeros(8), M) * (1.-P_cond_2)
    return(Pi_val)

# conditional gumbel expectation
def e(choice, P_cond):
    other = 1. - choice
    return(mu + np.log(1. + np.exp(pi(other,P_cond) - pi(choice,P_cond))))

In [31]:
# create transition probability function for given P
# order: (m1,0,0), (m1,1,0), (m1,0,1), (m1,1,1), (m2,0,0), (m2,1,0), (m2,0,1), (m2,1,1)

def F(P_cond):
    # flip indices for second player
    P_cond_2 = P_cond.copy()
    P_cond_2[2], P_cond_2[1], P_cond_2[6], P_cond_2[5] = P_cond_2[1], P_cond_2[2], P_cond_2[5], P_cond_2[6]
    
    P_1 = np.column_stack([1.-P_cond, 
                           P_cond, 
                           1.-P_cond, 
                           P_cond])
    P_2 = np.column_stack([1.-P_cond_2, 
                           1.-P_cond_2, 
                           P_cond_2, 
                           P_cond_2])
    big_P_1 = np.column_stack([P_1,P_1])
    big_P_2 = np.column_stack([P_2,P_2])
    
    big_B = np.repeat(np.repeat(B, 4, axis = 0), 4, axis = 1)

    return(big_P_1 * big_P_2 * big_B)

In [32]:
def Gamma(P_cond):
    inv_term = np.linalg.inv(np.eye(8) - beta * F(P_cond))
    e_1 = e(1., P_cond)
    e_0 = e(0., P_cond)
    sum_term = P_cond * (pi(1., P_cond) + e_1) + (1.-P_cond) * e_0
    return(inv_term @ sum_term.reshape(-1,1))

In [62]:
def Psi(P_cond):
    gamma_vals = Gamma(P_cond)
    probs = F(P_cond)
    
    in_probs = probs.copy()
    in_probs[:,np.where(X[:,0] == 0.)] = 0.
    
    out_probs = probs.copy()
    out_probs[:,np.where(X[:,0] == 1.)] = 0.
    
    cond_prob_in = in_probs / np.sum(in_probs, axis = 1)
    cond_prob_out = out_probs / np.sum(out_probs, axis = 1)
    
    e_1 = e(1., P_cond).reshape(-1,1)
    e_0 = e(0., P_cond).reshape(-1,1)
        
    V_1 = pi(1., P_cond).reshape(-1,1) + e_1 + beta * cond_prob_in @ gamma_vals
    V_0 = pi(0., P_cond).reshape(-1,1) + e_0 + beta * cond_prob_out @ gamma_vals
        
    print(V_1)
    print(V_0)
    P_update = np.exp(V_1) / (np.exp(V_0) + np.exp(V_1))
    
    
    return(P_update.reshape(-1))

In [63]:
converged = False
crit = 1e-6
P_cond_old = np.ones(8) / 2.
while not converged:
    P_cond_new = Psi(P_cond_old)
    print(P_cond_new)
    if np.max(np.abs(P_cond_new - P_cond_old)) <= crit:
        converged = True
    else:
        P_cond_old = P_cond_new

[[36.36583659]
 [37.2050213 ]
 [36.36583659]
 [37.2050213 ]
 [36.94968508]
 [38.10253611]
 [36.94968508]
 [38.10253611]]
[[35.41961962]
 [36.25880432]
 [35.41961962]
 [36.25880432]
 [35.97366981]
 [37.12652084]
 [35.97366981]
 [37.12652084]]
[0.72035374 0.72035374 0.72035374 0.72035374 0.72631684 0.72631684
 0.72631684 0.72631684]
[[34.25692059]
 [34.93875915]
 [34.25692059]
 [34.93875915]
 [34.96923105]
 [35.9924247 ]
 [34.96923105]
 [35.9924247 ]]
[[33.94445825]
 [34.62629681]
 [33.94445825]
 [34.62629681]
 [33.74040636]
 [34.76360001]
 [33.74040636]
 [34.76360001]]
[0.57748618 0.57748618 0.57748618 0.57748618 0.7736128  0.7736128
 0.7736128  0.7736128 ]
[[30.26097246]
 [31.04500401]
 [30.26097246]
 [31.04500401]
 [39.38812908]
 [40.38128676]
 [39.38812908]
 [40.38128676]]
[[47.98248944]
 [48.76652098]
 [47.98248944]
 [48.76652098]
 [27.97662691]
 [28.96978459]
 [27.97662691]
 [28.96978459]]
[0.001 0.001 0.001 0.001 0.999 0.999 0.999 0.999]
[[   18.92719269]
 [   20.07952678]
 [   18

<ipython-input-62-54aa865322e6>:24: RuntimeWarning: overflow encountered in exp
  P_update = np.clip(np.exp(V_1) / (np.exp(V_0) + np.exp(V_1)), eps, 1.-eps)
<ipython-input-62-54aa865322e6>:24: RuntimeWarning: invalid value encountered in true_divide
  P_update = np.clip(np.exp(V_1) / (np.exp(V_0) + np.exp(V_1)), eps, 1.-eps)



[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan

 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]

...

 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[nan nan nan nan nan nan nan nan]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]

KeyboardInterrupt: 